In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-community langchain-openai tavily-python

In [2]:
import getpass
import os
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict
from langchain import hub
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from typing import Union
from typing import Literal
from langgraph.graph import END
from langgraph.graph import StateGraph, START

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

In [3]:
tools = [TavilySearchResults(max_results=3)]

In [40]:
prompt = hub.pull("ih/ih-react-agent-executor")
prompt.pretty_print()

llm = ChatOpenAI(model="gpt-4o")
agent_executor = create_react_agent(llm, tools, state_modifier=prompt)

C:\Users\ArhamShams\anaconda3\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are a helpful assistant.

============================= Messages Placeholder =============================

{messages}


In [41]:
class PlanExecute(TypedDict):
    input: str
    experience:str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

## Planning Step

In [49]:
class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

In [50]:
planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Given a list of skills and years of experience, generate a detailed contexts for quizzes based on provided skills and years of experience. Your goal is to create a context that evaluates the depth of knowledge and expertise that someone with the specified experience level is expected to have. \
            Each context should provide comprehensive and unique information about the skills, core fundamentals of the skill, practical applications in industry scenarios its core fundamentals,challenges typically faced and how they are resolved,Best practices and patterns relevant to the skill. Refrain from adding any Source URL in the context.  \
            Ensure the context is clear, precise, and factual. You are tasked to come up with a simple step-by-step plan on how you can create a context. \
            This plan should involve individual tasks, that if executed correctly, will yield the correct answer. Do not add any superfluous steps. \
            The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.""",
        ),
        ("placeholder", "{skillset}"),
        ("placeholder", "{experience}"),
    ]
)

planner = planner_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(Plan)

In [51]:
planner.invoke(
    {
        "skillset": [("user", "{Angular}")],
        "experience": [("user", "5")], 
    }
)

Plan(steps=['Identify the core fundamentals of Angular, such as components, modules, services, directives, and dependency injection.', 'Research practical applications of Angular in industry scenarios, such as building single-page applications, progressive web apps, and enterprise-level applications.', 'Identify common challenges faced by developers with 5 years of experience in Angular, such as state management, performance optimization, and handling large-scale applications.', 'Research solutions and best practices for resolving these challenges, including the use of state management libraries like NgRx, lazy loading for performance optimization, and modular architecture for scalability.', 'Compile best practices and design patterns relevant to Angular, such as the use of reactive programming with RxJS, component-based architecture, and the use of Angular CLI for efficient development.', 'Synthesize the information into a comprehensive context that evaluates the depth of knowledge an

Re-Plan Step

In [52]:
class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


replanner_prompt = ChatPromptTemplate.from_template(
    """Given a list of skills and years of experience, generate a detailed contexts for quizzes based on provided skills and years of experience. Your goal is to create a context that evaluates the depth of knowledge and expertise that someone with the specified experience level is expected to have. \
            Each context should provide comprehensive and unique information about the skills, core fundamentals of the skill, practical applications in industry scenarios its core fundamentals,challenges typically faced and how they are resolved,Best practices and patterns relevant to the skill. Refrain from adding any Source URL in the context \
            Ensure the context is clear, precise, and factual. You are tasked to come up with a simple step-by-step plan on how you can create a context. \
            This plan should involve individual tasks, that if executed correctly, will yield the correct answer. Do not add any superfluous steps. \
            The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)


replanner = replanner_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(Act)

## Create the Graph

In [53]:
async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    agent_response = await agent_executor.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"skillset": [("user", state["input"])],
                                  "experience":[("user",state["experience"])]
                                 })
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "agent"

In [54]:
workflow = StateGraph(PlanExecute)

workflow.add_node("planner", plan_step)

workflow.add_node("agent", execute_step)

workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

workflow.add_edge("planner", "agent")

workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    should_end,
    ["agent", END],
)

app = workflow.compile()

In [55]:
config = {"recursion_limit": 50}
inputs = {
    "input": "{Angular}",
    "experience": "{5}",  # Added number_of_experience
}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

{'plan': ['Identify the core fundamentals of Angular, such as components, modules, services, directives, and dependency injection.', 'Research practical applications of Angular in industry scenarios, such as building single-page applications, progressive web apps, and enterprise-level applications.', 'Explore common challenges faced by developers with 5 years of experience in Angular, such as state management, performance optimization, and handling large-scale applications.', 'Investigate how these challenges are typically resolved, including the use of tools like NgRx for state management, lazy loading for performance optimization, and best practices for structuring large applications.', 'Identify best practices and design patterns relevant to Angular, such as the use of reactive programming with RxJS, component-based architecture, and the use of Angular CLI for efficient development.', 'Compile the gathered information into a comprehensive context that evaluates the depth of knowledg

{'plan': ['Investigate how these challenges are typically resolved, including the use of tools like NgRx for state management, lazy loading for performance optimization, and best practices for structuring large applications.', 'Identify best practices and design patterns relevant to Angular, such as the use of reactive programming with RxJS, component-based architecture, and the use of Angular CLI for efficient development.', 'Compile the gathered information into a comprehensive context that evaluates the depth of knowledge and expertise expected from someone with 5 years of experience in Angular.']}
{'past_steps': [('Investigate how these challenges are typically resolved, including the use of tools like NgRx for state management, lazy loading for performance optimization, and best practices for structuring large applications.', "Here's a summary of findings regarding how challenges in Angular development are typically resolved using tools like NgRx for state management, lazy loading